In [9]:
from pymilvus import AnnSearchRequest, WeightedRanker, RRFRanker
# 测试向量搜索单元
# from langchain_community.embeddings import OllamaEmbeddings
# from sentence_transformers import SentenceTransformer
from pymilvus import FieldSchema, CollectionSchema, DataType, MilvusClient
from FlagEmbedding import FlagModel
from langchain_community.document_transformers import LongContextReorder
import numpy as np

client = MilvusClient(uri="http://192.168.200.130:19530")

def emb_text(texts):
    # embedding_model = SentenceTransformer('maidalun1020/bce-embedding-base_v1')
    # embedding_vectors = embedding_model.encode(text, batch_size=1024)
    embedding_model = FlagModel('chuxin-llm/Chuxin-Embedding', use_fp16=True)
    embedding_vectors = embedding_model.encode_queries(texts, batch_size=256)

    # embedding_vectors = normalize(embedding_vectors, norm='l2')

    return embedding_vectors

doc = ["什么时候举行庆祝矿大115周年篮球友谊赛"]

doc_vector = emb_text(doc)

search_params = {
    "metric_type": "COSINE",
    "params": {}
}

# IVF_FLAT
# res = client.search(
#     collection_name="cumt_gpt_qwen",
#     data=doc_vector,
#     limit=10,
#     output_fields=["id", "url", "content", "publish_time"],
#     search_params=search_params
# )

# HNSW
# res = client.search(
#   collection_name="cumt_gpt_chuxin", # Collection name
#   data=doc_vector, # Replace with your query vector
#   search_params={
#     "metric_type": "COSINE",
#     "params": {"ef":150, "radius":0.36}, # Search parameters
#   }, # Search parameters
#   limit=10, # Max. number of search results to return
#   output_fields=["id","url","content","publish_time"], # Fields to return in the search results
#   consistency_level="Bounded"
# )

res = client.hybrid_search(
    collection_name="cumt_gpt_chuxin",
    reqs=[
        AnnSearchRequest(
            data=doc_vector,  # 查询向量
            anns_field="content_vector",  # 搜索字段
            param={"metric_type": "COSINE", "params": {"ef":150, "radius":0.36}},  # 搜索参数
            limit=5,  # 返回前10个结果
            # expr="publish_time > 1627776000"  # 过滤条件
        ),
        AnnSearchRequest(
            data=doc_vector,  # 查询向量
            anns_field="title_vector",  # 搜索字段
            param={"metric_type": "COSINE", "params": {"ef":150, "radius":0.36}},  # 搜索参数
            limit=5,  # 返回前10个结果
            # expr="publish_time > 1627776000"  # 过滤条件
        )
    ],
    # ranker=WeightedRanker(float(0.7), float(0.3)),
    ranker=RRFRanker(),
    limit=5,
    output_fields=["id", "url", "content", "title", "publish_time"],
    timeout=60,
    consistency_level="Bounded"
    # partition_names: Optional[List[str]] = None
)

reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(res[0])


print(res)
print("----------------")
print(reordered_docs)

data: ["[{'id': 453316957169519928, 'distance': 0.03177805617451668, 'entity': {'id': 453316957169519928, 'url': 'https://news.cumt.edu.cn/../info/1002/70129.htm', 'content': '中国矿业大学举行建校115周年创新发展大会编辑：李居铭发布时间：2024-06-02\\n\\n大会现场\\n\\n中国工程院院士钱七虎教授作主旨报告\\n\\n中国工程院院士陈清泉教授作主旨报告\\n\\n中国工程院院士谢和平教授作主旨报告\\n\\n中国工程院院士赵跃民教授作主旨报告\\n\\n校友代表、中国工程院院士康红普发言\\n\\n教师代表、德国国家科学与工程院院士、建筑与设计学院教师伯恩哈德·穆勒教授发言\\n\\n徐州市市长王剑锋讲话\\n\\n校党委书记刘波主持大会\\n\\n校长宋学锋致辞\\n\\n徐州市副市长刘景、副校长李强代表签署新一轮校地战略合作协议\\n\\n教师代表、化工学院董良教授发言\\n\\n在校学生代表、孙越崎学院2020级本科生陈硕发言\\n\\n大会现场\\n百十五载薪火相传，砥砺奋进共谱新篇。6月1日上午，中国矿业大学庆祝建校115周年创新发展大会在南湖校区力行楼明德礼堂隆重举行。\\n大会气氛简朴热烈，校党委书记刘波主持大会，校长宋学锋作校庆报告。出席大会的两院院士、知名专家有中国工程院院士钱七虎、中国工程院院士陈清泉、中国工程院院士谢和平、中国工程院院士康红普、中国工程院院士赵跃民、原中科院生态中心副主任杨敏、安徽建筑大学副校长蔡国军、武汉大学学术委员会副主任刘泉声；海外专家学者有德国国家科学与工程院院士伯恩哈德·穆勒，美国工程院院士德里克·埃尔斯沃斯，美国工程院院士杜山·博罗耶维奇，新加坡工程院院士梁春辉，加拿大工程院院士杨瑞林，越南河内地矿大学副校长赵雄常，澳大利亚伍伦贡大学教授任廷祥；徐州市委副书记、市长王剑锋，副市长刘景；校友嘉宾有中国煤炭地质总局航测遥感局局长赖百炼，西安音乐学院党委书记张立杰，徐州工程学院院长张农，新疆工程学院党委副书记、院长王增国，徐矿集团副总经理、总工程师陈清华。各级来宾、学校老领导、现任校领导、教师代表、学生代表和校友代表等共计1200余